In [1]:
import os;
os.getcwd()

'/Users/dan/Documents/Finance/Programs/Financial Machine Learning/Backtesting'

In [2]:
import pandas as pd
from tqdm import tqdm
import sys
from datetime import datetime
# custom imports from this repo
sys.path.append("../Financial Data Structures")
from utils.read_data import read_ticks

In [3]:
ticks = "/Users/dan/Documents/Finance/Historical Data/ES-Futures-Ticks-20230807-20240806.trades.filtered.csv"
# data = read_ticks(ticks, nrows=None, date_handling="epoch")

process_date = lambda x: datetime.fromisoformat(x).timestamp()

t = 0
with open(ticks, "r") as f:
    for i, line in enumerate(f):
        if i == 1:
            line = line.strip().split(",")
            t = process_date(line[0])
            break

skipped = 0 
data = {"time": [], "price": [], "volume": [], "symbol": []}
with open(ticks, "r") as f:
    for i, line in enumerate(tqdm(f, total=82971690)):
        if i == 0:
            continue
        line = line.strip().split(",")
        time = process_date(line[0])
        if time < t:
            skipped += 1
            continue
        t += 0.25
        data["time"].append(process_date(line[0]))
        data["price"].append(float(line[1]))
        data["volume"].append(float(line[2]))
        data["symbol"].append(line[3])


82971691it [01:37, 855195.98it/s]                              


In [5]:
# Todo: Apply 1 tick of slippage to each trade

from collections import deque
import numpy as np

window = 600
prices = deque(maxlen=window)
upper_band = 0
lower_band = 0
mu = 0
std = 0
qty = 0
open_price = 0
current_profit = 0
total_profit = 0
commission = 2.25
leverage = 50
trade_cost = commission * 2 / leverage


buys = []
sells = []
holding_periods = []

for i in tqdm(range(len(data['time']))):
    last_price = data['price'][i]
    prices.append(last_price)
    if len(prices) < window:
        continue

    mu = np.mean(prices)
    std = np.std(prices)

    upper_band = mu + 2 * std
    lower_band = mu - 2 * std

    if qty == 1:
        current_profit = (last_price - open_price) * leverage - trade_cost
    elif qty == -1:
        current_profit = (open_price - last_price) * leverage - trade_cost

    if qty == 0: 
        if last_price < lower_band - trade_cost:
            open_price = last_price
            qty = 1
            mean_side = "under"
            buys.append(i)

        elif last_price > upper_band + trade_cost:
            open_price = last_price
            qty = -1
            mean_side = "over"
            sells.append(i)

    elif qty == 1:
        if last_price >= mu and mean_side == "under":
            open_price = last_price
            qty = 0
            mean_side = "over"
            total_profit += current_profit
            sells.append(i)
            holding_periods.append(sells[-1] - buys[-1])

    elif qty == -1:
        if last_price <= mu and mean_side == "over":
            open_price = last_price
            qty = 0
            mean_side = "under"
            total_profit += current_profit
            buys.append(i)
            holding_periods.append(buys[-1] - sells[-1])


f"{total_profit=}\n{np.mean(holding_periods)=}"




100%|██████████| 82181655/82181655 [1:05:07<00:00, 21033.37it/s]


'total_profit=1383684.1199983445\nnp.mean(holding_periods)=334.76931275642056'

In [6]:
# plot the prices with markers for buys and sells
import matplotlib.pyplot as plt
plt.plot(data['price'])
plt.scatter(buys, [data['price'][i] for i in buys], color='green', marker='triangle_up')
plt.scatter(sells, [data['price'][i] for i in sells], color='red', marker='triangle_down')
plt.show()

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x104c34470>>
Traceback (most recent call last):
  File "/opt/miniconda3/envs/finance/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
